In [1]:
# import required modules
import requests
from bs4 import BeautifulSoup
import csv

AIF = []

pages = []
for i in range(2):
    page = 'https://wikileaks.org/irq/sort/category/supporting_aif_' +str(i) +'.html'
    pages.append(page)

print pages

def get_page_data(page):
    #Get Request
    req = requests.get(page)

    # read the content of the server’s response
    src = req.text

    # parse the response into an HTML tree
    soup = BeautifulSoup(src)

    #pull links to events within table
    soup_table = (soup.select('table.report td.c2'))
    
    num = len(soup_table)
    
    # store links in list
    links = []
    for i in range(num):
        links.append((str(soup_table[i]).split("\""))[3])
    
    # store link ids in list
    IDs = []
    for i in range(num):
        link = links[i]
        id = str(link.split("/")[5]).split(".")[0]
        IDs.append(id)

    #extract year and month from link
    yearmonth = []
    for i in range(num):
        yearmonth.append(str(links[i])[12:19])
    
    # match links with their event ids
    events = []
    for i in range(num):
        tup = (IDs[i], links[i], yearmonth[i])
        events.append(tup)
    
#at this point, we have pulled the links and ids of each AIF event
#now we want to run the link through a function to scrape each event page for lat, long, date 

    # store the links in a form we can use to webscrape each individual event page
    urls = []
    for i in range(num):
        pull = events[i][1]
        urls.append('http://wikileaks.org' + pull)

    # webscrape latitude, longitude, and event description from each page
    set_list = []
    for url in urls:
        src_events = requests.get(url).text
        soup_events = BeautifulSoup(src_events)
        line2 = str(soup_events.select('table.r2 td'))
        pull_cardinals = line2.split(">")
        pull_long = pull_cardinals[11]
        pull_lat = pull_cardinals[9]
        set = (str(pull_lat).split("<")[0], str(pull_long).split("<")[0])
        set_list.append(set)

    AIFevent_list = []
    for i in range(num):
        lat = set_list[i][0]
        long = set_list[i][1]
        info = (events[i][0], events[i][1], events[i][2], lat, long)
        AIFevent_list.append(info)
    
    for i in range(num):
        AIF.append(AIFevent_list[i])

#put lists together in one long list   
for i in pages:
    print i
    get_page_data(i)        
    print AIF

#write CSV, since it is a list, we will need to add the headers in r    
with open("AIFoutput.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(AIF)

['https://wikileaks.org/irq/sort/category/supporting_aif_0.html', 'https://wikileaks.org/irq/sort/category/supporting_aif_1.html']
https://wikileaks.org/irq/sort/category/supporting_aif_0.html
[('IRQ20041026n827', '/irq/report/2004/10/IRQ20041026n827.html', '2004/10', '33.210495', '44.36906052'), ('IRQ20050727n2219', '/irq/report/2005/07/IRQ20050727n2219.html', '2005/07', '33.28689957', '44.06549072'), ('IRQ20050808n2337', '/irq/report/2005/08/IRQ20050808n2337.html', '2005/08', '35.42382813', '44.33744049'), ('IRQ20050926n2568', '/irq/report/2005/09/IRQ20050926n2568.html', '2005/09', '34.59077072', '43.67869186'), ('IRQ20051013n2826', '/irq/report/2005/10/IRQ20051013n2826.html', '2005/10', '34.44541168', '43.79758835'), ('IRQ20051013n2836', '/irq/report/2005/10/IRQ20051013n2836.html', '2005/10', '31.72332954', '46.10445023'), ('IRQ20051015n2800', '/irq/report/2005/10/IRQ20051015n2800.html', '2005/10', '36.3837204', '42.45209885'), ('IRQ20051015n2820', '/irq/report/2005/10/IRQ20051015n2

In [2]:
len(AIF)

70